In [1]:
# imports
#%matplotlib widget

import psycopg
import matplotlib as mp
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

import ipywidgets as widgets
from IPython.display import display

In [2]:
db_config = {'user':'jeffrey',
                  'password':'strawberries',
                  'host':'127.0.0.1',
                  'port':'5432',
                  'dbname':'trader',
                  'autocommit':True} 

# connect to psql database
#conn = psycopg.connect(host="localhost", dbname="trader", user="jeffrey", password="strawberries")
conn = psycopg.connect(**db_config)
cur = conn.cursor()

today = dt.datetime.today()                 # datetime.datetime object
yesterday = today - dt.timedelta(days=1)    # datetime.datetime object
data_date = today                           # datetime.datetime object

In [3]:
def symbol_qry():
    query = "SELECT DISTINCT symbol FROM reqid_list;"
    cur.execute(query)
    unique_symbols = cur.fetchall()
    return unique_symbols

def dates_qry(symbol):
    query = """
        SELECT DISTINCT DATE(send_time) FROM reqid_list WHERE symbol = %s;
    """
    cur.execute(query, (symbol,))
    unique_dates = cur.fetchall()
    unique_dates.sort()
    return [date[0] for date in unique_dates]

In [4]:
# Extract symbols from unique_symbols
unique_symbols = symbol_qry()
symbols = [symbol[0] for symbol in unique_symbols]

# Create a dropdown widget
symbol_dropdown = widgets.Dropdown(
    options=symbols,
    value=symbols[0],
    description='Symbol:',
    disabled=False,
)

dates = dates_qry(symbol_dropdown.value)

# Create a date dropdown widget
date_dropdown = widgets.Dropdown(
    options=[''] + dates,
    value=dates[0],
    description='Date:',
    disabled=False,
)

def update_dates(*args):
    new_dates = dates_qry(symbol_dropdown.value)
    date_dropdown.options = [''] + new_dates
    date_dropdown.value = new_dates[0] if new_dates else ''

symbol_dropdown.observe(update_dates, 'value')

display(symbol_dropdown)
display(date_dropdown)



Dropdown(description='Symbol:', options=('AAPL', 'MSFT', 'DELL', 'NFLX', 'AVGO', 'TSM', 'SPX', 'LLY', 'NVDA', …

Dropdown(description='Date:', index=1, options=('', datetime.date(2024, 7, 1), datetime.date(2024, 7, 2), date…

In [5]:
def get_data_from_tick_price(symbol, date):
    query = """
        SELECT
            r.source,
            r.reqid,
            r.send_time,
            r.req_func,
            r.symbol,
            r.security_type,
            r.id,
            t.source,
            t.reqid,
            t.recv_time,
            t.field,
            t.name,
            t.price,
            t.attributes,
            t.id
        FROM
            reqid_list r
        JOIN
            tick_price t
        ON
            r.reqid = t.reqid
        WHERE
            r.symbol = %s AND r.send_time::date = %s AND t.recv_time::date = %s
        ORDER BY t.recv_time ASC;
    """
    cur.execute(query, (symbol, date, date))
    data = cur.fetchall()
    return data


In [6]:
def get_data_tbt_all_last(symbol, date):
    try:
        query = """
            SELECT
                r.source,
                r.reqid,
                r.send_time,
                r.req_func,
                r.symbol,
                r.security_type,
                r.id,
                t.source,
                t.reqid,
                t.recv_time,
                t.tick_type,
                t.tick_name,
                t.ib_time,
                t.price,
                t.size,
                t.exchange,
                t.id
            FROM
                reqid_list r
            JOIN
                tbt_all_last t
            ON
                r.reqid = t.reqid
            WHERE
                r.symbol = %s AND r.send_time::date = %s AND t.recv_time::date = %s
            ORDER BY t.recv_time ASC;
        """
        cur.execute(query, (symbol, date, date))
        data = cur.fetchall()
        return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        pass

In [8]:
def get_reqid_and_symbol_today(today):
    query = """
        SELECT reqid, symbol
        FROM reqid_list
        WHERE DATE(send_time) = %s;
    """
    cur.execute(query, (today.date(),))
    reqid_and_symbol = cur.fetchall()
    return reqid_and_symbol

reqid_and_symbol_today = get_reqid_and_symbol_today(today)
print(reqid_and_symbol_today)

[(103, 'SPX'), (202, 'AMD'), (302, 'NVDA'), (402, 'MSFT'), (502, 'NFLX'), (602, 'TSM'), (702, 'AMZN'), (802, 'DELL'), (902, 'INTC'), (1002, 'MU'), (1102, 'WDC'), (1202, 'AVGO'), (1302, 'AAPL'), (1402, 'SPY'), (1502, 'PANW'), (1602, 'LLY'), (1702, 'SMCI'), (1802, 'SMH'), (103, 'SPX'), (202, 'AMD'), (302, 'NVDA'), (402, 'MSFT'), (502, 'NFLX'), (602, 'TSM'), (702, 'AMZN'), (802, 'DELL'), (902, 'INTC'), (1002, 'MU'), (1102, 'WDC'), (1202, 'AVGO'), (1302, 'AAPL'), (1402, 'SPY'), (1502, 'PANW'), (1602, 'LLY'), (1702, 'SMCI'), (1802, 'SMH'), (99202, 'AMD')]


In [7]:
def get_unique_reqid_tbt_all_last():
    query = """
        SELECT DISTINCT reqid
        FROM tbt_all_last;
    """
    cur.execute(query)
    unique_reqid = cur.fetchall()
    return [reqid[0] for reqid in unique_reqid]

# Example usage
unique_reqid_tbt_all_last = get_unique_reqid_tbt_all_last()
print(unique_reqid_tbt_all_last)

[203, 303, 403, 503, 603, 703, 1203, 1403, 9203, 9303, 9403, 9503, 91403, 99203]


In [ ]:
def get_all_tbt_all_last(reqid, date):
    query = """
        SELECT
            t.reqid,
            t.source,
            t.reqid,
            t.recv_time,
            t.tick_type,
            t.tick_name,
            t.ib_time,
            t.price,
            t.size,
            t.exchange,
            t.id
        FROM
            tbt_all_last t
        WHERE
            t.reqid = %s AND t.recv_time::date = %s
        ORDER BY
            t.recv_time ASC;
    """
    cur.execute(query, (reqid, date))
    print(reqid, date)
    prices = cur.fetchall()
    return prices

# Example usage
#prices = get_all_tbt_all_last(symbol_dropdown.value, date_dropdown.value)
prices = get_all_tbt_all_last(203, date_dropdown.value)
print(prices)

In [ ]:
def get_reqid_and_symbol_by_date(date):
    query = """
        SELECT reqid, symbol, send_time
        FROM reqid_list
        WHERE DATE(send_time) = %s;
    """
    cur.execute(query, (date,))
    reqid_and_symbol = cur.fetchall()
    return reqid_and_symbol

# Example usage
reqid_and_symbol_by_date = get_reqid_and_symbol_by_date(date_dropdown.value)
print(reqid_and_symbol_by_date)

In [ ]:
reqid_and_symbol_by_date_simple = [(reqid, symbol, send_time.strftime('%Y-%m-%d')) for reqid, symbol, send_time in reqid_and_symbol_by_date]

print(reqid_and_symbol_by_date_simple)

In [ ]:
def get_reqid_and_symbol(yesterday):
    query = """
        SELECT reqid, symbol
        FROM reqid_list
        WHERE DATE(send_time) = %s;
    """
    cur.execute(query, (date_dropdown.value,))
    reqid_and_symbol = cur.fetchall()
    return reqid_and_symbol

reqid_and_symbol_yesterday = get_reqid_and_symbol(yesterday)
print(reqid_and_symbol_yesterday)

In [ ]:
data_price = get_data_from_tick_price(symbol_dropdown.value, date_dropdown.value)
# Define the column names based on the structure of stock_data
price_columns = ['source_r', 'reqid_r', 'send_time', 'req_func', 'symbol', 'security_type', 'id_r', 
           'source_t', 'reqid_t', 'recv_time', 'field', 'name', 'price', 'attributes', 'id_t']

# Convert stock_data to a pandas DataFrame
data_price_df = pd.DataFrame(data_price, columns=price_columns)

#data_price_df = data_price_df.sort_values(by='recv_time')

# Display the DataFrame
print(data_price_df.head())

In [ ]:
data_tbt = get_data_tbt_all_last(symbol_dropdown.value, date_dropdown.value)

tbt_columns = ['source_r', 'reqid_r', 'send_time_r', 'req_func_r', 'symbol_r', 'security_type_r', 'id_r',
            'source_t', 'reqid_t', 'recv_time_t', 'tick_type_t', 'tick_name_t', 'ib_time_t', 'price_t',
            'size_t', 'exchange_t', 'id_t']            

# Convert stock_data to a pandas DataFrame
data_tbt_df = pd.DataFrame(data_tbt, columns=tbt_columns)

#data_tbt_df = data_tbt_df.sort_values(by='recv_time')

# Display the DataFrame
print(data_tbt_df.head())


In [ ]:
print(data_df.dtypes)

In [ ]:
data_df['recv_time'] = data_df['recv_time'].dt.time

In [ ]:
print(data_df.dtypes)

In [ ]:
data_df['price'] = data_df['price'].astype(float)
#print(data_df.dtypes)

In [ ]:
print(data_df.dtypes)

In [ ]:
print(data_df.head())

In [ ]:
last_price_data = data_df[data_df['name'] == 'LAST']
bid_price_data = data_df[data_df['name'] == 'BID']
ask_price_data = data_df[data_df['name'] == 'ASK']

# print(last_price_data.head())
# print(bid_price_data.head())
# print(ask_price_data.head())


In [ ]:
print(last_price_data.dtypes)

In [ ]:
last_price_data.to_csv('ibtrader_last_price_data.csv', index=False)

In [ ]:
# Convert recv_time to datetime if not already
#last_price_data['recv_time'] = pd.to_datetime(last_price_data['recv_time'])

print(last_price_data['recv_time'].iloc[0])

type(last_price_data['recv_time'].iloc[0])

#last_price_data['recv_time'].diff()


In [ ]:
# Calculate the time difference between consecutive rows
last_price_data['time_diff'] = last_price_data['recv_time'].diff().dt.total_seconds()

last_price_data['time_diff'] = last_price_data['recv_time'].diff()

# Identify the indices where the time difference is greater than 1 second
gap_indices = last_price_data.index[last_price_data['time_diff'] > 1].tolist()


In [ ]:
test = last_price_data['recv_time']

In [ ]:
delta_t = test.diff()



In [ ]:
print(test.head(20))

In [ ]:
# Split the data into chunks based on the gap indices
chunks = []
prev_index = 0
for index in gap_indices:
    chunks.append(last_price_data.iloc[prev_index:index])
    prev_index = index
chunks.append(last_price_data.iloc[prev_index:])  # Add the last chunk

# Remove the time_diff column
last_price_data.drop(columns=['time_diff'], inplace=True)

# Display the number of chunks and the first few rows of each chunk
print(f"Number of chunks: {len(chunks)}")
for i, chunk in enumerate(chunks):
    print(f"\nChunk {i+1}:")
    print(chunk.head())

In [ ]:
# df1 = data_df.iloc[::2].reset_index(drop=True)  # Rows with even indices
# df2 = data_df.iloc[1::2].reset_index(drop=True)  # Rows with odd indices

# print("DataFrame 1:")
# print(df1.head())
# print("\nDataFrame 2:")
# print(df2.head())

In [ ]:
# df1.to_csv('df1.csv', index=False)
# df2.to_csv('df2.csv', index=False)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df1['recv_time'], df1['price'], label='Price')
plt.xlabel('Receive Time')
plt.ylabel('Price')
plt.title('Price over Time')
plt.legend()
plt.grid(True)

plt.get_current_fig_manager().toolbar.pan()
plt.get_current_fig_manager().toolbar.zoom()

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(last_price_data['recv_time'], last_price_data['price'], marker='o', markersize=4, markerfacecolor='purple', label='Price')
#ax.scatter(last_price_data['recv_time'], last_price_data['price'], marker='.', label='Price')
plt.show()